In [1]:
base_folder = "/mnt/e/SteamLibrary2/steamapps/common/SlayTheSpire/SpeedTheSpire/DEBUG"

In [2]:
import tensorflow as tf

# modified from code from https://machinelearningmastery.com/lstm-autoencoders/

# lstm autoencoder reconstruct and predict sequence
from numpy import array
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model

2023-05-12 15:50:47.578808: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 15:50:47.623833: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-12 15:50:47.625065: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 15:50:48.520880: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# https://stackoverflow.com/a/3207973
from os import listdir
from os.path import isfile, join
datafiles = [f for f in listdir(base_folder) if isfile(join(base_folder, f))]
len(datafiles)

103

In [4]:
corpus = []

for datafilename in datafiles:
    fullpath = join(base_folder, datafilename)
    with open(fullpath, "r") as datafile:
        lines = datafile.readlines()
        for line in lines:
            if line[0] == "I":
                corpus.append(line[1:-1])
len(corpus)

8919

In [5]:
corpus = [[char for char in line] for line in corpus]

In [6]:
import string

alphabet = " ?%!.$\\:_-(){}[]\",+" + string.ascii_lowercase + string.ascii_uppercase + string.digits
char_to_idx = {c: alphabet.find(c) for c in alphabet}
print(char_to_idx)

{' ': 0, '?': 1, '%': 2, '!': 3, '.': 4, '$': 5, '\\': 6, ':': 7, '_': 8, '-': 9, '(': 10, ')': 11, '{': 12, '}': 13, '[': 14, ']': 15, '"': 16, ',': 17, '+': 18, 'a': 19, 'b': 20, 'c': 21, 'd': 22, 'e': 23, 'f': 24, 'g': 25, 'h': 26, 'i': 27, 'j': 28, 'k': 29, 'l': 30, 'm': 31, 'n': 32, 'o': 33, 'p': 34, 'q': 35, 'r': 36, 's': 37, 't': 38, 'u': 39, 'v': 40, 'w': 41, 'x': 42, 'y': 43, 'z': 44, 'A': 45, 'B': 46, 'C': 47, 'D': 48, 'E': 49, 'F': 50, 'G': 51, 'H': 52, 'I': 53, 'J': 54, 'K': 55, 'L': 56, 'M': 57, 'N': 58, 'O': 59, 'P': 60, 'Q': 61, 'R': 62, 'S': 63, 'T': 64, 'U': 65, 'V': 66, 'W': 67, 'X': 68, 'Y': 69, 'Z': 70, '0': 71, '1': 72, '2': 73, '3': 74, '4': 75, '5': 76, '6': 77, '7': 78, '8': 79, '9': 80}


In [7]:
for line in corpus:
    for char in line:
        if char not in alphabet:
            assert char in alphabet

In [8]:
def one_hot_encode(line):
    result = []
    for char in line:
        row = [0] * len(alphabet)
        row[char_to_idx[char]] = 1
        result.append(row)
    return result

def idx_encode(line):
    result = [char_to_idx[char] for char in line]
    return result

def sparse_tensor_encode(line):
    assert False, "TODO: implement sparse_tensor_encode"
    result = tf.sparse.SparseTensor(indices=[],
                                    values = [],
                                    dense_shape=[len(line), len(alphabet)])
    return result

In [9]:
len(corpus[4])

10463

In [10]:
def chop_and_mask(line, limit=10000, padding=-1):
    new_line = line[:limit]
    if len(new_line) < limit:
        new_line += [padding] * (limit - len(new_line))
    return new_line

In [11]:
sum([len(line) for line in corpus]) / len(corpus)

9596.274694472475

In [12]:
STATE_MAX_LENGTH = 12000

In [13]:
# define input sequence

seq_in = [chop_and_mask(idx_encode(line), limit=STATE_MAX_LENGTH) for line in corpus]

In [16]:
# adapted from https://blog.keras.io/building-autoencoders-in-keras.html

timesteps = STATE_MAX_LENGTH  # Length of your sequences
input_dim = 1
latent_dim = 500

inputs = Input(shape=(timesteps, input_dim))
encoded = LSTM(latent_dim)(inputs)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)


2023-05-12 15:52:07.203037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-12 15:52:07.204142: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-12 15:52:07.205216: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [18]:
sequence_autoencoder.compile(optimizer='adam', loss='mse')
sequence_autoencoder.fit([seq_in[0]], [seq_in[0]], epochs=300, verbose=2)


Epoch 1/300


2023-05-12 15:52:34.344866: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-12 15:52:34.345885: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-12 15:52:34.347145: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 - 31s - loss: 673.4733 - 31s/epoch - 31s/step
Epoch 2/300
1/1 - 32s - loss: 665.6948 - 32s/epoch - 32s/step
Epoch 3/300
1/1 - 35s - loss: 662.8562 - 35s/epoch - 35s/step
Epoch 4/300
1/1 - 37s - loss: 657.0008 - 37s/epoch - 37s/step
Epoch 5/300
1/1 - 31s - loss: 646.4641 - 31s/epoch - 31s/step
Epoch 6/300
1/1 - 38s - loss: 645.5232 - 38s/epoch - 38s/step
Epoch 7/300
1/1 - 48s - loss: 645.3761 - 48s/epoch - 48s/step
Epoch 8/300
1/1 - 50s - loss: 645.3463 - 50s/epoch - 50s/step
Epoch 9/300


KeyboardInterrupt: 

In [ ]:
def old_code():# reproduction of inputs attempt
    num_samples = len(seq_in)

    model = Sequential()
    model.add(LSTM(100, activation='relu', input_shape=(len(seq_in[0]),1)))
    model.add(RepeatVector(n_in))
    model.add(LSTM(100, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(array(seq_in[0]), array(seq_in[0]), epochs=300, verbose=2)
    plot_model(model, show_shapes=True, to_file='reconstruct_lstm_autoencoder.png')
    # demonstrate recreation
    yhat = model.predict(sequence, verbose=0)
    print(yhat[0,:,0])